In [ ]:
import pandas as pd

data = pd.read_csv('strokeData.csv')

#check for null 
data.isnull().sum()


In [ ]:
"""fill missing values for bmi using median as more accurate, as mean skewed
by extreme data and mode can be skewed by outliers
"""
data['bmi'].fillna(data['bmi'].median(), inplace=True)
data.head()

#count how many stroke and non stroke for class balance
strokes = data['stroke'].value_counts()
#print(strokes)

'''
as the class imbalance is huge SMOTE is needed to artificially balance it out
'''

In [ ]:
#drop non essential data that would be used to predict stroke, use new data frame
dataFiltered = data.copy()
#stroke dropped as we are predicting and id is identifier and no related
dataFiltered.drop(['id'],axis=1,inplace=True)
'''encode the data as categorical data is not allowed in machine learning
hot encoding as no order or hierarchy
'''
dataFiltered = pd.get_dummies(dataFiltered, columns=['gender','ever_married','work_type','Residence_type','smoking_status'])

#correlation test to see what to drop
dataFiltered.corr()


In [ ]:
#target and input data
predictors = dataFiltered.drop(columns=['stroke'])
target = dataFiltered['stroke']

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Apply SMOTE before train-test split
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(predictors, target)

#split data into test and train data
X_train,X_test,y_train,y_test = train_test_split(X_resampled,y_resampled,test_size=0.3,random_state=42)




In [ ]:
#using hyper parameters - gridSearchcv for best resutls
from sklearn.model_selection import GridSearchCV # type: ignore
from sklearn.ensemble import RandomForestClassifier # type: ignore

param_grid = {
    'n_estimators' : [100,200,300,400,500],
    'max_depth': [None,10,20,30],
}


#fit model using gridsearch
#cv = cross validation, spliting data into training and validation sets multiple times in diff ways
#verbose - level of feedback
grid = GridSearchCV(estimator=rf, param_grid=param_grid,cv=5,scoring='accuracy', n_jobs=-1,verbose=1)

grid.fit(X_train_resampled,y_train_resampled)

print(grid.best_params_)
best_params = grid.best_params_


In [ ]:
best_score = grid.best_score_
print("Best Cross-Validation Accuracy:", best_score)



In [ ]:
#implementing a better forest model using those best parameters
optimizedRf = RandomForestClassifier(**best_params)

#training model
optimizedRf.fit(X_train,y_train)
#prediciting model
y_prediction = optimizedRf.predict(X_test)
optimizedAccuracy = accuracy_score(y_test, y_prediction)
optimizedR_squared = rf.score(X_test,y_train)

print(optimizedAccuracy)
print(optimizedR_squared)
